Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [1]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-cp37-none-any.whl size=19877 sha256=de95a4349761544b35d2a458c7132a5c84160e3c882f3ad4fb986b580d957a93
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
from google.colab import drive
drive.mount('drive')

In [2]:
import os
import pathlib
import numpy as np
import pandas as pd
import scipy.stats
import tensorflow as tf
import joblib
from tqdm.notebook import tqdm
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F
import torch
from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [3]:
#If CUDA isn't set up properly for Tensorflow, then at least maximize the number of threads available for CPU
if not tf.config.list_physical_devices('GPU'):  # No GPU(s) found
    tf.config.threading.set_inter_op_parallelism_threads(2)
    tf.config.threading.set_intra_op_parallelism_threads(os.cpu_count() // 2)

In [4]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [5]:
seed_1 = 1
rand_1 = np.random.seed(seed_1)
seed_2 = rand_1
rand_2 = np.random.seed(seed_2)
seed_3 = rand_2
rand_3 = np.random.seed(seed_3)

In [6]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [8]:
%%time

print("# Loading data...")

NUMERAI_S3_BUCKET_URL = "https://numerai-public-datasets.s3-us-west-2.amazonaws.com"

#read in the example predictions from local storage
#EXAMPLE_PREDS = 'tournament_predictions.csv'

#or downlod the example predictions from Numerai's S3 bucket:
EXAMPLE_PREDS_URL = NUMERAI_S3_BUCKET_URL + "/latest_numerai_example_predictions_data.csv.xz"

#download the latest tournament data file:
TOURNAMENT_DATA_URL = NUMERAI_S3_BUCKET_URL + "/latest_numerai_tournament_data.csv.xz"

###IMPORTANT! DELETE THE FILE BELOW IF YOU CHANGE MODELS! OTHERWISE, RENAME THE FILE FOR YOUR VARIOUS MODELS###
LM_CACHE_FILE = pathlib.Path("neutralization.cache.joblib")

#read-in or download the example predictions
exp_df = pd.read_csv(EXAMPLE_PREDS_URL, index_col=0)
#download the tournament data
tournament_data = pd.read_csv(TOURNAMENT_DATA_URL, index_col=0)

#merge them together
full_df = pd.merge(tournament_data, exp_df, left_index=True, right_index=True)

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 1min 14s, sys: 3.73 s, total: 1min 18s
Wall time: 1min 22s


In [9]:
feature_names = [f for f in full_df.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [10]:
%%time
X = full_df[feature_names]

CPU times: user 2.28 s, sys: 780 ms, total: 3.06 s
Wall time: 3.06 s


In [13]:
X

,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,feature_charisma27,feature_charisma28,...,feature_wisdom7,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,0.50,0.25,...,0.00,0.00,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00
n000920ed083903f,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,0.00,0.75,...,0.50,0.50,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50
n0038e640522c4a6,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,0.50,0.50,...,0.00,0.25,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00
n004ac94a87dc54b,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,0.00,0.75,...,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25
n0052fe97ea0c05f,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,0.50,0.00,...,1.00,0.00,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffe8c032b4

In [11]:
%%time
Y = full_df[full_df.columns[-1]]

CPU times: user 171 µs, sys: 23 µs, total: 194 µs
Wall time: 198 µs


In [12]:
Y

id
n0003aa52cab36c2    0.49
n000920ed083903f    0.49
n0038e640522c4a6    0.53
n004ac94a87dc54b    0.51
n0052fe97ea0c05f    0.50
                    ... 
nffe8c032b43dd80    0.50
nffed97510bc5f0f    0.50
nffedc372225a6eb    0.50
nfff1707ac2823b5    0.48
nfff5fa87a5f9eec    0.50
Name: prediction, Length: 1671309, dtype: float64

In [ ]:
'''
le = LabelEncoder()
Y_enc = le.fit_transform(Y)
'''

In [ ]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [14]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=rand_2)

model_x = [
          
    LGBMRegressor(random_state=rand_1),
    LGBMRegressor(random_state=rand_2),
    LGBMRegressor(random_state=rand_3)
]

from sklearn.metrics import explained_variance_score

S_train, S_test = stacking(model_x,                   
                          X_train, y_train, X_test,   
                          regression=True, 
                          mode='oof_pred_bag',
                          needs_proba=False,
                          save_dir=None,
                          metric= explained_variance_score,
                          n_folds=7,
                          stratified=True,
                          shuffle=True,
                          random_state=rand_3,verbose=2)


del model_x

gc.collect()

/usr/local/lib/python3.7/dist-packages/vecstack/core.py:461: UserWarning: This is regression task hence classification-specific parameters set to <True> were ignored: <stratified>
  warnings.warn(warn_str, UserWarning)


task:         [regression]
metric:       [explained_variance_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LGBMRegressor]
    fold  0:  [0.76404054]
    fold  1:  [0.76414157]
    fold  2:  [0.76413051]
    fold  3:  [0.76201572]
    fold  4:  [0.76224472]
    fold  5:  [0.76331008]
    fold  6:  [0.76337211]
    ----
    MEAN:     [0.76332218] + [0.00082039]
    FULL:     [0.76332342]

model  1:     [LGBMRegressor]
    fold  0:  [0.76383508]
    fold  1:  [0.76306916]
    fold  2:  [0.75999283]
    fold  3:  [0.76393289]
    fold  4:  [0.76334618]
    fold  5:  [0.76298181]
    fold  6:  [0.76235496]
    ----
    MEAN:     [0.76278756] + [0.00124418]
    FULL:     [0.76278873]

model  2:     [LGBMRegressor]
    fold  0:  [0.76312241]
    fold  1:  [0.76400332]
    fold  2:  [0.76317349]
    fold  3:  [0.76333837]
    fold  4:  [0.76323894]
    fold  5:  [0.76283666]
    fold  6:  [0.76167737]
    ----
    MEAN:     [0.76305580] + [0.00065220]
    FULL:     [0.7

In [15]:
model_e = LGBMRegressor(random_state=rand_1)
gc.collect()

517

In [17]:
model_1 = LGBMRegressor(random_state=rand_1)
model_1.fit(X[feature_names], Y)
gc.collect()

579

In [18]:
model_2 = LGBMRegressor(random_state=rand_2)
model_2.fit(X[feature_names], Y)
gc.collect()

32

In [19]:
model_3 = LGBMRegressor(random_state=rand_3)
model_3.fit(X[feature_names], Y)
gc.collect()

80

In [20]:
y_pred_model_1 = model_1.predict(X[feature_names])
gc.collect()

54

In [21]:
y_pred_model_2 = model_2.predict(X[feature_names])
gc.collect()

57

In [22]:
y_pred_model_3 = model_3.predict(X[feature_names])
gc.collect()

54

In [23]:
df_mix = pd.DataFrame(columns=['f0','f1','f2'])
df_mix['f0'] = y_pred_model_1
df_mix['f1'] = y_pred_model_2
df_mix['f2'] = y_pred_model_3

In [24]:
df_mix

,f0,f1,f2
0,0.489630,0.489630,0.489630
1,0.491946,0.491946,0.491946
2,0.521446,0.521446,0.521446
3,0.506467,0.506467,0.506467
4,0.495433,0.495433,0.495433
...,...,...,...
1671304,0.502685,0.502685,0.502685
1671305,0.498112,0.498112,0.498112
1671306,0.511050,0.511050,0.511050
1671307,0.482753,0.482753,0.482753


In [25]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

./numerai_dataset_255.zip: 395MB [00:08, 49.4MB/s]                           
2021-03-14 10:40:34,104 INFO numerapi.base_api: unzipping file...


In [26]:
%%time
print("# Loading data...")
tournament_df = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 50.5 s, sys: 12.1 s, total: 1min 2s
Wall time: 1min 2s


In [27]:
model_e = model_e.fit(S_train, y_train)
y_pred = model_e.predict(S_test)
gc.collect()

83

In [28]:
y_pred_final = model_e.predict(df_mix)
tournament_df[PREDICTION_NAME] = y_pred_final
gc.collect()

54

In [29]:
tournament_df[PREDICTION_NAME]

id
n0003aa52cab36c2    0.487912
n000920ed083903f    0.490317
n0038e640522c4a6    0.525922
n004ac94a87dc54b    0.507797
n0052fe97ea0c05f    0.494724
                      ...   
nffe8c032b43dd80    0.502953
nffed97510bc5f0f    0.497857
nffedc372225a6eb    0.513111
nfff1707ac2823b5    0.479350
nfff5fa87a5f9eec    0.493808
Name: prediction, Length: 1671309, dtype: float64

In [30]:
import numerapi
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

In [31]:
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)

In [33]:
tournament_df[PREDICTION_NAME]

id
n0003aa52cab36c2    0.487912
n000920ed083903f    0.490317
n0038e640522c4a6    0.525922
n004ac94a87dc54b    0.507797
n0052fe97ea0c05f    0.494724
                      ...   
nffe8c032b43dd80    0.502953
nffed97510bc5f0f    0.497857
nffedc372225a6eb    0.513111
nfff1707ac2823b5    0.479350
nfff5fa87a5f9eec    0.493808
Name: prediction, Length: 1671309, dtype: float64

In [40]:
data_rfe_10 = reduce_all_exposures(tournament_df,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.1)

era121 
era122 
era123 
era124 
era125 
era126 
era127 
era128 
era129 
era130 
era131 
era132 
era575 
era576 
era577 
era578 
era579 
era580 
era581 
era582 
era583 
era584 
era585 
era586 
era587 
era588 
era589 
era590 
era591 
era592 
era593 
era594 
era595 
era596 
era597 
era598 
era599 
era600 
era601 
era602 
era603 
era604 
era605 
era606 
era607 
era608 
era609 
era610 
era611 
era612 
era613 
era614 
era615 
era616 
era617 
era618 
era619 
era620 
era621 
era622 
era623 
era624 
era625 
era626 
era627 
era628 
era629 
era630 
era631 
era632 
era633 
era634 
era635 
era636 
era637 
era638 
era639 
era640 
era641 
era642 
era643 
era644 
era645 
era646 
era647 
era648 
era649 
era650 
era651 
era652 
era653 
era654 
era655 
era656 
era657 
era658 
era659 
era660 
era661 
era662 
era663 
era664 
era665 
era666 
era667 
era668 
era669 
era670 
era671 
era672 
era673 
era674 
era675 
era676 
era677 
era678 
era679 
era680 
era681 
era682 
era683 
era684 
era685 
era686 
era687 


In [41]:
# replace prediction with reduced feature exposure prediction and rescale to [0,1]
tournament_df[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
tournament_df[PREDICTION_NAME] -= tournament_df[PREDICTION_NAME].min()
tournament_df[PREDICTION_NAME] /= tournament_df[PREDICTION_NAME].max()

In [42]:
tournament_df[PREDICTION_NAME]

id
n0003aa52cab36c2    0.464095
n000920ed083903f    0.427727
n0038e640522c4a6    0.511523
n004ac94a87dc54b    0.741013
n0052fe97ea0c05f    0.468786
                      ...   
nffe8c032b43dd80    0.505329
nffed97510bc5f0f    0.339889
nffedc372225a6eb    0.691348
nfff1707ac2823b5    0.428819
nfff5fa87a5f9eec    0.540247
Name: prediction, Length: 1671309, dtype: float64

In [43]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [44]:
tournament_df[PREDICTION_NAME].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

In [45]:
# NameOfYourAI FISAGOL_2
public_id = "7TISUDGAWEVO2B35ECOQQXU2RWXGZN3I"
secret_key = "QJYUWIMFEEDNZ4GHUO6VSSKPMRCBFJIMJ7BZ65ESIWRN4YHGYHSRJDNL64TAG7EH"
model_id = "c6f3339f-f677-48a0-ac42-2995afc614a5"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [46]:
print("Uploading DataFrame in Numerai...")
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)
print("DataFrame Uploaded...")

2021-03-14 17:12:04,242 INFO numerapi.base_api: uploading predictions...


Uploading DataFrame in Numerai...
DataFrame Uploaded...


In [ ]:
tournament_data[PREDICTION_NAME].to_csv('FisaGol1_1.csv')
!cp FisaGol1_1.csv "drive/My Drive/PHOENIXSIGMA/"

In [ ]:
'''
##########################################################################################
##############################   SAVE MODEL   ############################################
##########################################################################################
from sklearn.externals import joblib
joblib.dump(my_model, "my_model.pkl")
# and later...
my_model_loaded = joblib.load("my_model.pkl")
'''

'\n##########################################################################################\n##############################   SAVE MODEL   ############################################\n##########################################################################################\nfrom sklearn.externals import joblib\njoblib.dump(my_model, "my_model.pkl")\n# and later...\nmy_model_loaded = joblib.load("my_model.pkl")\n'